## Imports

In [34]:
import os
import pandas as pd
import numpy as np

# import streamlit as st
#langchain imports
from langchain import HuggingFaceTextGenInference, HuggingFaceHub, HuggingFacePipeline
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from ollama import chat

In [3]:
# !pip install ollama

In [144]:
messages = [
    {
        'role': 'user',
        'content': 'why is the sky blue?'
    }
]

In [5]:
for part in chat('llama3', messages=messages, stream=True):
    print(part['message']['content'], end='', flush=True)

What a great question!

The sky appears blue to our eyes because of a phenomenon called Rayleigh scattering, which is named after the British physicist Lord Rayleigh, who first described it in the late 19th century.

Here's what happens:

1. When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases like nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of light.
2. The shorter wavelengths of light, such as blue and violet, are scattered more efficiently by these small molecules than the longer wavelengths, like red and orange.
3. As a result, the blue and violet light is distributed throughout the atmosphere in all directions, reaching our eyes from all parts of the sky.
4. Meanwhile, the longer wavelengths of light, like red and orange, continue to travel in more direct paths to our eyes, with less scattering, giving them a more direct route.

This is why the sky appears blue during the daytime: our eyes are seeing the scattered b

In [8]:
from openai import OpenAI

In [14]:
OpenAI?

Init signature:
OpenAI(
    *,
    api_key: 'str | None' = None,
    organization: 'str | None' = None,
    project: 'str | None' = None,
    base_url: 'str | httpx.URL | None' = None,
    timeout: 'Union[float, Timeout, None, NotGiven]' = NOT_GIVEN,
    max_retries: 'int' = 2,
    default_headers: 'Mapping[str, str] | None' = None,
    default_query: 'Mapping[str, object] | None' = None,
    http_client: 'httpx.Client | None' = None,
    _strict_response_validation: 'bool' = False,
) -> 'None'
Docstring:     
Abstract base class for generic types.

A generic type is typically declared by inheriting from
this class parameterized with one or more type variables.
For example, a generic mapping type might be defined as::

  class Mapping(Generic[KT, VT]):
      def __getitem__(self, key: KT) -> VT:
          ...
      # Etc.

This class can then be used as follows::

  def lookup_name(mapping: Mapping[KT, VT], key: KT, default: VT) -> VT:
      try:
          return mapping[key]
      exc

## Using OpenApi api

In [86]:
client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

In [90]:
response = client.chat.completions.create(
  model="llama3",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The LA Dodgers won in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)
print(response.choices[0].message.content)

The 2020 World Series was played at Globe Life Field in Arlington, Texas.


## Langchain

In [91]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [22]:
llm = Ollama(
    model='llama3',
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler])
)

In [39]:
# llm("Tell me 5 things about the Roman history")
for c in  llm.stream("Tell me 5 things about the Roman history"):
    print(c, end='', flush=True)

Here are five key points about Roman history:

1. **The Founding of Rome**: According to legend, Rome was founded in 753 BC by Romulus and Remus, two brothers who were abandoned as infants and suckled by a she-wolf. This mythological founding date is still celebrated today on April 21st, the Feast of Romulus and Remus.

2. **The Roman Republic**: From its founding until 27 BC, Rome was a republic governed by a system of elected representatives and senators. The Roman Republic expanded throughout Italy, conquered much of Europe and North Africa, and developed a complex system of laws and governance.

3. **The Punic Wars**: Rome's greatest rival in the ancient Mediterranean was Carthage, a city-state in modern-day Tunisia. Three wars between Rome and Carthage (264-146 BC) saw the rise of legendary Roman generals like Hannibal and Scipio Africanus, and ultimately led to the destruction of Carthage and the dominance of Rome.

4. **The Roman Empire**: In 27 BC, the Roman Senate granted Augu

## Prompt Template

In [43]:
from langchain_core.prompts import PromptTemplate

In [168]:
prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template='I want to open a restaurant for {cuisine} food. Suggest me a fancy name for it, just one. Just the name and nothing else'
)

prompt_template_name.format(cuisine="Nepali")

'I want to open a restaurant for Nepali food. Suggest me a fancy name for it, just one. Just the name and nothing else'

In [169]:
name_chain = prompt_template_name | llm #| StrOutputParser()
# chain1 = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

# for c in  name_chain.stream("Nepali"):
#     print(c, end='', flush=True)

In [170]:
prompt_template_items = PromptTemplate(
    input_variables=['restaurant_name'],
    template="Suggest some 10 menu items for {restaurant_name}. Return the menu items as comma separated values and nothing else."
)

In [171]:
items_chain = prompt_template_items | llm #| StrOutputParser()

# for c in  items_chain.stream("Himalayan Haat"):
#     print(c, end='', flush=True)

## Connect Prompts/Chains

In [188]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.output_parsers import JsonOutputToolsParser

In [174]:
chain = {'restaurant_name': name_chain} | items_chain | StrOutputParser()

for c in  chain.stream("Nepali"):
    print(c, end='', flush=True)

chain.invoke("Nepali")

Kathmandu Momo Platter, Himalayan Lamb Chops with Garlic Naan, Royal Thali with Goat Curry, Darjeeling Tea-Infused Chicken Tikka Masala, Pokhara Fish Fry with Lemon Butter Sauce, Maharaja's Mixed Vegetable Biryani, Pashmina Steak with Soy-Ginger Glaze, Yatra's Nepalese Cheese Fritters, Gurkha's Ginger-Turmeric Soup, and Shakti's Saffron-Infused Rice Pilaf.

'Momo Masala, Nepalese Lamb Curry, Chana Masala, Chicken Tikka Makhani, Vegetable Korma, Gorkhali Buffet, Thukpa Soup, Palpasa Tihar Rice Bowl, Everest Burger, Royal Thali'

### Passing the variable through chains

In [191]:
# Combine the chains
combined_chain = RunnableParallel(
    {
        "restaurant_name": name_chain,
        "cuisine": RunnablePassthrough(),
    }
).assign(
    menu_items=lambda x: items_chain.invoke(x)
)

# Run the combined chain
input_data = {"cuisine": "Italian"}
result = combined_chain.invoke(input_data)

# Print the results
print(f"Restaurant Name: {result['restaurant_name']}")
print(f"Menu Items: {result['menu_items']}")

Restaurant Name: "Bellavista"
Menu Items: Tuscan Shrimp Pasta, Pan-Seared Salmon, Vegan Quinoa Bowl, Margherita Pizza, Chicken Parmesan, Spaghetti Bolognese, Caprese Salad, Garlic Roasted Broccoli, Bruschetta Appetizer, Espresso Tiramisu


## Restaurant Name Generator Project RAG project
### ollama x langchain x streamlit x llm x RAG

![](images/screenshot.png){width=5000}

**Summary**
1. Generate a name of a restaurant based on the `name` of the cuisine - name - Italan, Mexian, Nepali, Indian etc.
2. Generate `10 menu items` based on the `name` of the restaurant

**Setup**
1. Ollama
    * Download and install Ollama - [Ollama Website](https://ollama.com/)
    * Download one of the models from the ollama and run it. I am using `llama3`
```
ollama pull llama3
```
run the ollama in background

2. Install the requirements
   ```
   pip install -r requirements.txt
   ```
3. Run the streamlit app
   ```
   streamlit run app.py
   ```

Note: This project is inspired from the Codebasic youtube channel tutorial.
